In [ ]:
# this notebook is "scratchpad" code, as part of feature inclusion/exclusion exploration

In [1]:
"""
Created on Wed Jun 19 11:48:32 2019

@author: Andrew Welsh

This notebook is for doing a model fit to the historical data, then save the fitted model via joblib. The data must 
be extracted via SQL prior to inclusion in this notebook (until a data connection directly to the database is configured),
then saved via CSV, which is read here. 

The parameter tuning for this model fit is accomplished with the churn_grid_search_parameter_tuning notebook

Intended use is production model re-fitting process every 6 months, as well as the initial model fit.

Output: saved model files
"""

import pandas as pd
import churn_common as p #imports F2 score function, model_fit function

# imported data already has nulls cleaned and unused columns dropped
# churn_cloud_model_training_fit.csv is output from 'churn_cloud_model_training_fit.py' python script
cloud = pd.read_csv('churn_cloud_model_training_fit.csv', encoding='ISO-8859-1')

print('Query complete')
pd.options.display.max_columns = None

Query complete


In [3]:
cloud['time_month_key'].max()

201906

In [ ]:
# prior testing 

#index column from CSV is imported as a separate column, labeled 'Unnamed: 0', therefore must be dropped before model fit
cloud.drop(['Unnamed: 0'], axis=1, inplace=True)
#cloud = cloud.iloc[1:3000, :].copy() #select first 3k rows for fast testing

#Months are hard-coded here, use df['time_month_key'].max() above to find max TMK, configure eval set to be 3 most recent 
#months with enough time to determine churn (6mo per definition)
#Example: max TMK 201812; eval data is max TMK:201812 - 6mo - 3mo = 201806. For 3 month window, 201804 to 201806 inclusive, 
# dedicated_train is -6mo earlier from min TMK in eval data, i.e. 201804 - 6 = 201710 and earlier

#min TMK is 201601 max TMK is 201906, as of Aug 6 2019 for churn cloud.

#define model training dataset, set to TMK <= max(TMK)-14 (see above notes for reasoning)
cloud_train = cloud.loc[cloud['time_month_key']<=201708]
cloud_train.set_index(['account_number','time_month_key'])

#hold out evaluation dataset of last 3 months
cloud_eval = cloud.loc[(cloud['time_month_key']>=201806) & (cloud['time_month_key']<=201808)]
cloud_eval.set_index(['account_number','time_month_key'])

#the table head for the eval data should output below. Most accounts should have 3 rows each (3 latest TMKs).

In [ ]:
#split datasets into low-med-hi revenue segments
#training dataset
cloud_train_low, cloud_train_mid, cloud_train_hi = p.split_df_revenue_segments(cloud_train, 'average_invoiced_last_12_months')

#eval dataset
cloud_eval_low, cloud_eval_mid, cloud_eval_hi = p.split_df_revenue_segments(cloud_eval, 'average_invoiced_last_12_months')

In [ ]:
#==================================================================================================================
#==================================================================================================================
# Resample minority class in training dataset
#==================================================================================================================
#==================================================================================================================


#for repeatable results, set seed equal to an integer; for random assignment, set seed to None
seed = 98

cloud_train_low_rus = p.AndrewsRandomUnderSampler(cloud_train_low, 'target', seed)
cloud_train_mid_rus = p.AndrewsRandomUnderSampler(cloud_train_mid, 'target', seed)
cloud_train_hi_rus = p.AndrewsRandomUnderSampler(cloud_train_hi, 'target', seed)

print('Minority class resampling complete')

In [ ]:
exclude_col = exclude_col = [
        'account_number'
        ,'time_month_key'
        ,'time_month_key_dt'
        ,'account_name'
        ,'average_invoiced_last_12_months'
        ,'revenue_segment'
        ,'avg_monthly_invoice_band'
        ,'churn_flag'
        ,'target'
        ,'pct_change'
        ,'first_churn_tmk'
        ,'total_invoiced_in_month'
        ,'average_invoiced_next_6_months'
        ,'num_months_last_12_months'
        ,'num_months_next_6_months'
        ,'month_order'
        ,'month_order_desc'
        ,'time_month_key_eo_last_month_dt'
        ,'account_type_cloud_uk'
        ,'account_segment'
        ,'account_sub_type'
 ]
              #features that had total gain less than 3.0
              
include_col = ['average_value_of_opportunities_last_3_months'
,'average_value_of_opportunities_last_6_months'
,'avg_invoiced_in_last_3_months_vs_prior_3_months'
,'avg_invoiced_in_last_6_months_vs_prior_6_months'
,'avg_mthly_num_of_invoiced_items_in_last_6_months'
,'avg_per_line_item_invoiced_in_last_6_months'
,'avg_unit_price_cloud_backup'
,'avg_unit_price_cloud_block_storage'
,'avg_unit_price_cloud_databases'
,'avg_unit_price_cloud_files'
,'avg_unit_price_cloud_load_balancer'
,'avg_unit_price_cloud_monitoring'
,'avg_unit_price_cloud_queues'
,'avg_unit_price_cloud_sites'
,'avg_unit_price_first_gen_servers'
,'avg_unit_price_next_gen_servers'
,'avg_unit_price_total_outgoing_bw'
,'change_in_survey_score'
,'does_account_have_cloud_block_storage'
,'does_account_have_cloud_files'
,'does_account_have_next_gen_servers'
,'how_many_units_cloud_backup'
,'how_many_units_cloud_block_storage'
,'how_many_units_cloud_databases'
,'how_many_units_cloud_files'
,'how_many_units_cloud_load_balancer'
,'how_many_units_cloud_monitoring'
,'how_many_units_cloud_queues'
,'how_many_units_cloud_sites'
,'how_many_units_first_gen_servers'
,'how_many_units_next_gen_servers'
,'how_many_units_total_outgoing_bw'
,'is_cloud_rackconnect_linked'
,'last_survey_rating_original'
,'last_survey_responseflag'
,'last_survey_score'
,'mi_mo'
,'mi_mo_core'
,'mi_mo_managed'
,'mi_mo_mi'
,'mi_mo_mo'
,'mi_mo_unassigned'
,'num_distinct_account_bdc'
,'num_distinct_account_billing_city'
,'num_distinct_account_billing_country'
,'num_distinct_account_billing_postal_code'
,'num_distinct_account_billing_state'
,'num_distinct_account_billing_street'
,'num_distinct_account_business_type'
,'num_distinct_account_geographic_location'
,'num_distinct_account_manager'
,'num_distinct_account_primary_contact'
,'num_distinct_account_region'
,'num_distinct_account_sla_type'
,'num_distinct_account_team_name'
,'num_opportunities_last_12_months'
,'num_opportunities_last_3_months'
,'num_opportunities_last_6_months'
,'num_opportunities_lost_last_3_months'
,'num_opportunities_lost_last_6_months'
,'num_opportunities_won_last_3_months'
,'num_opportunities_won_last_6_months'
,'num_opps_allow_quote_allowed'
,'num_opps_allow_quote_not_allowed'
,'num_opps_bucket_influence_marketing'
,'num_opps_bucket_influence_null'
,'num_opps_bucket_source_marketing'
,'num_opps_bucket_source_null'
,'num_opps_bucket_source_sales'
,'num_opps_category_cloud_net_revenue'
,'num_opps_category_migration'
,'num_opps_category_new'
,'num_opps_category_new_footprint'
,'num_opps_category_new_logo'
,'num_opps_category_null'
,'num_opps_category_upgrade'
,'num_opps_commission_role_null'
,'num_opps_commission_role_pay_commissions'
,'num_opps_competitors_in_house'
,'num_opps_competitors_null'
,'num_opps_competitors_other'
,'num_opps_contract_length_0'
,'num_opps_contract_length_1'
,'num_opps_contract_length_12'
,'num_opps_contract_length_24'
,'num_opps_contract_length_null'
,'num_opps_cvp_verified_false'
,'num_opps_cvp_verified_true'
,'num_opps_data_quality_description_all_opportunity_details_captur'
,'num_opps_data_quality_description_missing_amount'
,'num_opps_data_quality_description_missing_amount_lead_source'
,'num_opps_data_quality_description_missing_amount_next_steps'
,'num_opps_data_quality_description_missing_lead_source'
,'num_opps_data_quality_description_missing_lead_source_next_ste'
,'num_opps_data_quality_description_missing_next_steps'
,'num_opps_data_quality_score_100'
,'num_opps_data_quality_score_40'
,'num_opps_data_quality_score_60'
,'num_opps_data_quality_score_80'
,'num_opps_econnect_received_false'
,'num_opps_econnect_received_true'
,'num_opps_focus_area_amazon'
,'num_opps_focus_area_cloud_office'
,'num_opps_focus_area_dedicated'
,'num_opps_focus_area_null'
,'num_opps_focus_area_openstack_public'
,'num_opps_focus_area_tricore'
,'num_opps_forecastcategory_closed'
,'num_opps_forecastcategory_omitted'
,'num_opps_forecastcategory_pipeline'
,'num_opps_forecastcategoryname_closed'
,'num_opps_forecastcategoryname_omitted'
,'num_opps_forecastcategoryname_pipeline'
,'num_opps_iswon_false'
,'num_opps_iswon_true'
,'num_opps_leadsource_call_in'
,'num_opps_leadsource_chat'
,'num_opps_leadsource_null'
,'num_opps_leadsource_outbound'
,'num_opps_leadsource_partner_network'
,'num_opps_leadsource_site_submission'
,'num_opps_leadsource_unknown'
,'num_opps_live_call_false'
,'num_opps_market_source_no'
,'num_opps_market_source_null'
,'num_opps_market_source_yes'
,'num_opps_nutcase_deal_probability_0'
,'num_opps_on_demand_reconciled_false'
,'num_opps_on_demand_reconciled_true'
,'num_opps_pain_point_null'
,'num_opps_pain_point_other'
,'num_opps_pain_point_servicenow'
,'num_opps_probability_0'
,'num_opps_probability_100'
,'num_opps_probability_15'
,'num_opps_requested_products_hosting_only'
,'num_opps_requested_products_null'
,'num_opps_support_unit_email_apps'
,'num_opps_support_unit_enterprise'
,'num_opps_support_unit_null'
,'num_opps_support_unit_smb'
,'num_opps_support_unit_tricore'
,'num_opps_ticket_type_null'
,'num_opps_ticket_type_upgrade'
,'num_opps_typex_aws'
,'num_opps_typex_dedicated_private_cloud'
,'num_opps_typex_mail_contract_signup'
,'num_opps_typex_rackspace_cloud'
,'num_opps_typex_revenue_ticket'
,'num_opps_typex_tricore'
,'num_opps_what_did_we_do_well_null'
,'num_opps_what_did_we_do_well_solution_fit'
,'num_opps_why_did_we_lose_existing_opp_closed_via_ticket'
,'num_opps_why_did_we_lose_no_response'
,'num_opps_why_did_we_lose_null'
,'num_opps_why_did_we_lose_price'
,'num_opps_why_did_we_lose_project_abandoned'
,'num_opps_why_did_we_lose_unresponsive'
,'number_of_cloud_accounts'
,'number_of_customer_accounts'
,'ownership_other'
,'ownership_private'
,'ownership_public'
,'ownership_subsidiary'
,'ownership_unknown'
,'pct_of_invoice_cloud_backup'
,'pct_of_invoice_cloud_block_storage'
,'pct_of_invoice_cloud_databases'
,'pct_of_invoice_cloud_files'
,'pct_of_invoice_cloud_load_balancer'
,'pct_of_invoice_cloud_monitoring'
,'pct_of_invoice_cloud_queues'
,'pct_of_invoice_cloud_sites'
,'pct_of_invoice_first_gen_servers'
,'pct_of_invoice_next_gen_servers'
,'pct_of_invoice_total_outgoing_bw'
,'pct_opportunities_lost_last_3_months'
,'pct_opportunities_lost_last_6_months'
,'pct_opportunities_won_last_3_months'
,'pct_opportunities_won_last_6_months'
,'rating_detractor_to_detractor'
,'rating_detractor_to_passive'
,'rating_detractor_to_promoter'
,'rating_passive_to_detractor'
,'rating_passive_to_passive'
,'rating_passive_to_promoter'
,'rating_promoter_to_detractor'
,'rating_promoter_to_passive'
,'rating_promoter_to_promoter'
,'ratio_mthly_num_of_invoiced_items_in_last_3_months_vs_prior_3_months'
,'ratio_mthly_num_of_invoiced_items_in_last_6_months_vs_prior_6_months'
,'seasonal_control'
,'second_last_survey_rating_original'
,'second_last_survey_responseflag'
,'second_last_survey_score'
,'site_branch'
,'site_headquarters'
,'site_single_location'
,'tam_acct_ratio'
,'tam_changed'
,'tam_ratio_changed'
,'total_invoiced_in_last_3_months_vs_prior_3_months'
,'total_invoiced_in_last_6_months'
,'total_invoiced_in_last_6_months_vs_prior_6_months'
,'total_value_of_opportunities_last_3_months'
,'total_value_of_opportunities_last_6_months'
               ]
predictors = list(cloud_train.columns[~cloud_train.columns.isin(exclude_col)])
#predictors = list(cloud_train.columns[cloud_train.columns.isin(include_col)])

#create copy of the X and Y columns
x1_train_res_low = cloud_train_low_rus[predictors] #uses list of columns excluding target (predictors) to select columns for x_low
y1_train_res_low = cloud_train_low_rus['target']

x1_train_res_mid = cloud_train_mid_rus[predictors]
y1_train_res_mid = cloud_train_mid_rus['target']

x1_train_res_hi = cloud_train_hi_rus[predictors]
y1_train_res_hi = cloud_train_hi_rus['target']

print("train ready")

#create copy of the X and Y columns for eval data
x_eval_low = cloud_eval_low[predictors] #uses list of columns excluding target (predictors) to select columns for x_low
y1_eval_low = cloud_eval_low['target']

x_eval_mid = cloud_eval_mid[predictors]
y1_eval_mid = cloud_eval_mid['target']

x_eval_hi = cloud_eval_hi[predictors]
y1_eval_hi = cloud_eval_hi['target']

print("eval ready")

In [ ]:
#import libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score, roc_auc_score
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

import time
import datetime
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [ ]:
import churn_common as p

In [ ]:
#==================================================================================================================
#==================================================================================================================
#==================================================================================================================
#Model run 
#==================================================================================================================
#==================================================================================================================
#==================================================================================================================

xgb_final_low = XGBClassifier(
        base_score=0.5,
        booster='gbtree',
        colsample_bylevel=1,
        colsample_bytree=0.9,
        gamma=0, 
        learning_rate=0.01,
        max_delta_step=2, 
        max_depth=14, 
        min_child_weight=0, 
        missing=None,
        n_estimators=1500, 
        random_state=seed,
        reg_alpha=0.1,
        reg_lambda=1,
        scale_pos_weight=1,
        silent=True,
        subsample=0.7,
        eval_metric='auc',
        objective='binary:logistic',
        nthread=-1
        )

xgb_final_mid = XGBClassifier(
        base_score=0.5,
        booster='gbtree',
        colsample_bylevel=1,
        colsample_bytree=0.8,
        gamma=0, 
        learning_rate=0.005,
        max_delta_step=1, 
        max_depth=11, 
        min_child_weight=1, 
        missing=None,
        n_estimators=1000, 
        random_state=seed,
        reg_alpha=0.01,
        reg_lambda=0.1,
        scale_pos_weight=1,
        subsample=0.7,
        verbosity=0,
        eval_metric='auc',
        objective='binary:logistic',
        nthread=-1
        )

xgb_final_hi = XGBClassifier(
        base_score=0.5, 
        booster='gbtree', 
        colsample_bylevel=1,
        colsample_bytree=0.7, 
        eval_metric='auc', 
        gamma=0,
        learning_rate=0.01, 
        max_delta_step=0, 
        max_depth=13, 
        max_features=13,
        min_child_weight=1, 
        missing=None, 
        n_estimators=1500, 
        n_jobs=1,
        nthread=-1, 
        objective='binary:logistic', 
        random_state=seed,
        reg_alpha=0.1, 
        reg_lambda=1e-05, 
        scale_pos_weight=1, 
        seed=seed,
        silent=True, 
        subsample=0.8
        )

p.modeleval(xgb_final_low, x1_train_res_low, y1_train_res_low.values.ravel(), 
         x_eval_low, y1_eval_low, predictors)
p.modeleval(xgb_final_mid, x1_train_res_mid, y1_train_res_mid.values.ravel(), 
         x_eval_mid, y1_eval_mid, predictors)
p.modeleval(xgb_final_hi, x1_train_res_hi, y1_train_res_hi.values.ravel(),
         x_eval_hi, y1_eval_hi, predictors)

#previous F2 scores: .525, .629, .644 (L, M, H)
#next: .526, .622, .643 (with gridsearched min_child_weight, max_depth, max_delta_step)

In [ ]:
#Available importance_types = ['weight', 'gain', 'cover', 'total_gain', 'total_cover']
#'gain' is the average GINI gain for that feature when used in a split

fi_low_gain = pd.DataFrame(xgb_final_low.get_booster().get_score(importance_type= 'gain'),index=[0]) #have to specify index to satisfy conversion to dataframe
fi_low_gain = fi_low_gain.T #transpose result
fi_low_gain = fi_low_gain.sort_values([0], ascending=[True]).copy()
fi_low_gain.to_excel('1feat_imp_low.xlsx')

fi_mid_gain = pd.DataFrame(xgb_final_mid.get_booster().get_score(importance_type= 'gain'),index=[0]) #have to specify index to satisfy conversion to dataframe
fi_mid_gain = fi_mid_gain.T #transpose result
fi_mid_gain = fi_mid_gain.sort_values([0], ascending=[True]).copy()
fi_mid_gain.to_excel('1feat_imp_mid.xlsx')

fi_hi_gain = pd.DataFrame(xgb_final_hi.get_booster().get_score(importance_type= 'gain'),index=[0]) #have to specify index to satisfy conversion to dataframe
fi_hi_gain = fi_hi_gain.T #transpose result
fi_hi_gain = fi_hi_gain.sort_values([0], ascending=[True]).copy()
fi_hi_gain.to_excel('1feat_imp_hi.xlsx')


In [ ]:
#based on list of features with gain <3.0 in the high-revenue accounts, eliminate those features from the dataset, 
#rerun the model

#examination of features from mid and low model for features with importance gain <3.0 show that their lists have features
#that are almost all in the elimination list from the high revenue accounts
#therefore, for simplicity in the first round, we'll eliminate the features from the high revenue accounts, as this list
#is much larger than the list from the mid and low rev. We'll check the model fit results and see if there's a negative impact

exclude_col = ['target'
               ,'total invoiced in last 6 months'
               ,'account_number'
               ,'time_month_key'
               ,'avg_6mo_eval'
               ,'avg_9mo_eval'
               ,'past_6mo_baseline'
#                ,'account_sub_type','account_segment'
#                ,'total invoiced in last 6 months vs prior 6 months'
#                ,'total invoiced in last 3 months vs prior 3 months'
#                ,'avg mthly num of invoiced items in last 6 months vs prior 6 months'
#                ,'avg mthly num of invoiced items in last 6 months'
#                ,'number of device_status = online/complete'
               
               #features that had total gain less than 3.0
,'does account have - rpc core'
,'rating_passive_to_passive_num'
,'num opps category - cloud net revenue'
,'rating_promoter_to_passive_num'
,'rating_passive_to_promoter_num'
,'num opps requested_products - hosting only'
,'how many units - rpc core'
,'does account have - faws'
,'opp_in_stage1_cnt_change_from_last_month'
,'num tickets ticket category - monitoring 1mo'
,'num tickets ticket category - monitoring alerts 1mo'
,'num opps typex - aws'
,'does account have - threat manager'
,'num distinct account_geographic_location'
,'avg minutes to first racker comment - category monitoring alerts 1mo'
,'opp_in_stage1_cnt'
,'avg minutes to started in progress - category monitoring alerts 1mo'
,'does account have - san'
,'num distinct account_business_type'
,'does account have - virtualization'
,'opp_in_stage3_cnt_change_from_last_month'
,'num distinct account_billing_country'
,'num opps data_quality_score - 100'
,'does account have - managed storage'
,'num distinct account_primary_contact'
,'pct of invoice - rpc core'
,'revenue_tickets_closed_last1mos'
,'num opps typex - mail contract signup'
,'avg days to close ticket - category customer initiated 1mo'
,'num distinct account_bdc'
,'num opportunities lost last 6 months'
,'num opportunities last 12 months'
,'num distinct account_team_name'
,'num opps focus_area - openstack public'
,'avg days to close ticket 1mo'
,'num opps leadsource - site submission'
,'does account have - bandwidth overages'
,'num opps leadsource - call in'
,'num opps focus_area - amazon'
,'num distinct account_billing_state'
,'does account have - rackspace email'
,'num tickets ticket category - application 1mo'
,'num distinct account_billing_city'
,'num opps contract_length - 1'
,'num opportunities lost last 3 months'
,'per unit price - rpc core'
,'opp_is_revenue_ticket_cnt_change_from_last_month'
,'does account have - load balancer'
,'num opps allow_quote - allowed'
,'num opps leadsource - partner network'
,'num tickets severity - emergency 1mo'
,'num opps forecastcategoryname - omitted'
,'how many units - managed exchange'
,'avg minutes to first racker comment - category monitoring alerts'
,'num opps category - new'
,'avg minutes to first racker comment 1mo'
,'num opps support_unit - smb'
,'rate of tickets confirmed solved 1mo'
,'avg days to close ticket - category application 1mo'
,'num tickets ticket category - operating services 1mo'
,'avg days to close ticket - all customer initiated tickets 1mo'
,'per unit price - virtual hosting'
,'num opps support_unit - email & apps'
,'avg days to close ticket - all racker initiated tickets 1mo'
,'num opps forecastcategoryname - pipeline'
,'avg minutes to first customer comment 1mo'
,'avg minutes to first racker comment'
,'num opps contract_length - 12'
,'num tickets ticket category - operating services'
,'num opps data_quality_description - missing: lead source'
,'num opps category - new logo'
,'num opps cvp_verified - true'
,'avg minutes to started in progress'
,'num opps probability - 0'
,'pct of devices with contract_status = in contract - risk of lapse 90 days'
,'num opps on_demand_reconciled - false'
,'num tickets ticket category - monitoring alerts'
,'avg days to close ticket - category hardware'
,'does account have - ppc'
,'num tickets ticket category - monitoring'
,'acc_tenure_in_months'
,'num opps data_quality_description - missing: next steps'
,'pct of revenue with contract_status = no contract status'
,'num distinct account_billing_postal_code'
,'does account have - mspc'
,'does account have - vmware'
,'avg days to close ticket - category customer initiated'
,'rate of tickets solved 1mo'
,'rating_promoter_to_promoter_num'
,'num opps focus_area - cloud office'
,'avg days to close ticket - category hardware 1mo'
,'does account have - cloud office'
,'avg days to close ticket'
,'num opps bucket_influence - marketing'
,'opp_is_revenue_ticket_cnt'
,'does account have - cloud san'
,'does account have - virtual hosting'
,'account_sub_type external customer'
,'how many units - cloud san'
,'does account have - switch'
,'avg minutes to started in progress 1mo'
,'avg days to close ticket - category application'
,'total number of tickets'
,'number of device_status = support maintenance'
,'num distinct account_billing_street'
,'num opps focus_area - cloud'
,'num tickets ticket category - account management 1mo'
,'num opps forecastcategory - omitted'
,'number of tickets solved 1mo'
,'num opps nutcase_deal_probability - 0'
,'per unit price - managed storage'
,'num opps cvp_verified - false'
,'num distinct account_manager'
,'num opps category - new footprint'
,'num tickets ticket category - application'
,'avg days to close ticket - all customer initiated tickets'
,'num opps data_quality_description - missing: amount'
,'per unit price - switch'
,'num opps data_quality_score - 60'
,'num opps probability - 100'
,'num opps market_source - yes'
,'num opps commission_role - pay commissions'
,'pct of invoice - switch'
,'rate of tickets confirmed solved'
,'num tickets ticket category - customer initiated 1mo'
,'num opps data_quality_score - 80'
,'num opps econnect_received - false'
,'second_last_survey_score'
,'pct of invoice - virtualization'
,'rate of tickets solved'
,'num tickets ticket category - account management'
,'pct of revenue with contract_status = in contract - risk of lapse 90 days'
,'number of tickets solved'
,'how many units - threat manager'
,'num opps data_quality_description - missing: amount, lead source'
,'num opps contract_length - 24'
,'num opps why_did_we_lose - project abandoned'
,'avg minutes to started in progress - category monitoring alerts'
,'number of devices os - firewall'
,'num tickets severity - emergency'
,'num opps contract_length - 0'
,'pct of invoice - managed storage'
,'avg minutes to started in progress - severity urgent 1mo'
,'num opps data_quality_description - all opportunity details captur'
,'does account have - managed exchange'
,'account_sub_type other'
,'per unit price - cloud san'
,'avg monthly number of - sku_description =  privatenet - 6 mth to 1 mth'
,'does account have - firewall'
,'num opps category - migration'
,'last_survey_score'
,'avg minutes to started in progress - severity urgent'
,'num opps leadsource - outbound'
,'num opps why_did_we_lose - price'
,'pct of invoice - load balancer'
,'num opps forecastcategory - pipeline'
,'num opps bucket_source - sales'
,'num distinct account_region'
,'num tickets ticket category - customer initiated'
,'num opps support_unit - enterprise'
,'num opportunities last 6 months'
,'avg days to close ticket - all racker initiated tickets'
,'num opps data_quality_description - missing: amount, next steps'
,'how many units - switch'
,'num opps ticket_type - upgrade'
,'number of device_status = online/complete'
,'num opps category - upgrade'
,'avg per line item invoiced in last 6 months_original'
              ]
include_col = ['opp_in_stage1_cnt'
    ,'opp_in_stage3_cnt'
    ,'opp_in_contract_state_cnt'
    ,'multi_stage_in_month_cnt'
    ,'opp_is_revenue_ticket_cnt'
    ,'opp_in_stage1_cnt_change_from_last_month'
    ,'opp_in_stage3_cnt_change_from_last_month'
    ,'opp_is_revenue_ticket_cnt_change_from_last_month'
    ,'website_chats_by_tmk'
    ,'website_calls_by_tmk'
    ,'website_sitesubs_by_tmk'
    ,'website_marketo_form_subs_by_tmk'
    ,'website_chats_by_tmk_change_from_last_month'
    ,'website_calls_by_tmk_change_from_last_month'
    ,'website_sitesubs_by_tmk_change_from_last_month'
    ,'website_marketo_form_subs_by_tmk_change_from_last_month'
    ,'avg credit memo in last 6 months'
    ,'total invoiced in last 3 months vs prior 3 months'
    ,'avg mthly num of invoiced items in last 3 months vs prior 3 months'
    ,'total number of tickets'
    ,'avg minutes to first customer comment'
    ,'avg minutes to first racker comment'
    ,'number of tickets confirmed solved'
    ,'number of tickets solved'
    ,'rate of tickets confirmed solved'
    ,'rate of tickets solved'
    ,'last_survey_score'
    ,'second_last_survey_score'
    ,'num opportunities last 6 months'
    ,'num opportunities won last 6 months'
    ,'num opportunities lost last 6 months'
    ,'pct opportunities won last 6 months'
    ,'pct opportunities lost last 6 months'
    ,'num opportunities last 3 months'
    ,'num opportunities won last 3 months'
    ,'num opportunities lost last 3 months'
    ,'pct opportunities won last 3 months'
    ,'pct opportunities lost last 3 months'
    ,'total value of opportunities last 6 months'
    ,'total value of opportunities last 3 months'
    ,'average value of opportunities last 6 months'
    ,'average value of opportunities last 3 months'
               ]
predictors = list(cloud_train.columns[~cloud_train.columns.isin(exclude_col)])
#predictors = list(cloud_num.columns[cloud_num.columns.isin(include_col)])

#create copy of the X and Y columns
x_low = cloud_train_low[predictors] #uses list of columns excluding target (predictors) to select columns for x_low
y1_low = cloud_train_low['target']

x_mid = cloud_train_mid[predictors]
y1_mid = cloud_train_mid['target']

x_hi = cloud_train_hi[predictors]
y1_hi = cloud_train_hi['target']

print("train ready")

#create copy of the X and Y columns for eval data
x_eval_low = cloud_eval_low[predictors] #uses list of columns excluding target (predictors) to select columns for x_low
y1_eval_low = cloud_eval_low['target']

x_eval_mid = cloud_eval_mid[predictors]
y1_eval_mid = cloud_eval_mid['target']

x_eval_hi = cloud_eval_hi[predictors]
y1_eval_hi = cloud_eval_hi['target']

print("eval ready")

#==================================================================================================================
#==================================================================================================================
# Resample minority class in training dataset
#==================================================================================================================
#==================================================================================================================

seed = 178

ros_low = RandomUnderSampler(random_state=seed)
x1_train_res_low0, y1_train_res_low0 = ros_low.fit_sample(x_low, y1_low)
x1_train_res_low = pd.DataFrame(x1_train_res_low0)
x1_train_res_low.columns = predictors
y1_train_res_low = pd.DataFrame(y1_train_res_low0)

ros_mid = RandomUnderSampler(random_state=seed)
x1_train_res_mid0, y1_train_res_mid0 = ros_mid.fit_sample(x_mid, y1_mid)
x1_train_res_mid = pd.DataFrame(x1_train_res_mid0)
x1_train_res_mid.columns = predictors
y1_train_res_mid = pd.DataFrame(y1_train_res_mid0)

ros_hi = RandomUnderSampler(random_state=seed)
x1_train_res_hi0, y1_train_res_hi0 = ros_hi.fit_sample(x_hi, y1_hi)
x1_train_res_hi = pd.DataFrame(x1_train_res_hi0)
x1_train_res_hi.columns = predictors
y1_train_res_hi = pd.DataFrame(y1_train_res_hi0)

print('Minority class resampling complete')

#rerun model using same parameters as before
p.modeleval(xgb_final_low, x1_train_res_low, y1_train_res_low.values.ravel(), 
         x_eval_low, y1_eval_low, predictors)
p.modeleval(xgb_final_mid, x1_train_res_mid, y1_train_res_mid.values.ravel(), 
         x_eval_mid, y1_eval_mid, predictors)
p.modeleval(xgb_final_hi, x1_train_res_hi, y1_train_res_hi.values.ravel(),
         x_eval_hi, y1_eval_hi, predictors)

In [ ]:
#Available importance_types = ['weight', 'gain', 'cover', 'total_gain', 'total_cover']
#'gain' is the average GINI gain for that feature when used in a split
#re-run for the first iteration of eliminated features
fi_low_gain = pd.DataFrame(xgb_final_low.get_booster().get_score(importance_type= 'gain'),index=[0]) #have to specify index to satisfy conversion to dataframe
fi_low_gain = fi_low_gain.T #transpose result
fi_low_gain = fi_low_gain.sort_values([0], ascending=[True]).copy()
fi_low_gain.to_excel('2feat_imp_low.xlsx')

fi_mid_gain = pd.DataFrame(xgb_final_mid.get_booster().get_score(importance_type= 'gain'),index=[0]) #have to specify index to satisfy conversion to dataframe
fi_mid_gain = fi_mid_gain.T #transpose result
fi_mid_gain = fi_mid_gain.sort_values([0], ascending=[True]).copy()
fi_mid_gain.to_excel('2feat_imp_mid.xlsx')

fi_hi_gain = pd.DataFrame(xgb_final_hi.get_booster().get_score(importance_type= 'gain'),index=[0]) #have to specify index to satisfy conversion to dataframe
fi_hi_gain = fi_hi_gain.T #transpose result
fi_hi_gain = fi_hi_gain.sort_values([0], ascending=[True]).copy()
fi_hi_gain.to_excel('2feat_imp_hi.xlsx')


In [ ]:
#2nd round of elimination
#based on list of features with gain <3.0 in the high-revenue accounts, eliminate those features from the dataset, 
#rerun the model

#examination of features from mid and low model for features with importance gain <3.0 show that their lists have features
#that are almost all in the elimination list from the high revenue accounts
#therefore, for simplicity in the first round, we'll eliminate the features from the high revenue accounts, as this list
#is much larger than the list from the mid and low rev. We'll check the model fit results and see if there's a negative impact

exclude_col = ['target'
               ,'total invoiced in last 6 months'
               ,'account_number'
               ,'time_month_key'
               ,'avg_6mo_eval'
               ,'avg_9mo_eval'
               ,'past_6mo_baseline'
#                ,'account_sub_type','account_segment'
#                ,'total invoiced in last 6 months vs prior 6 months'
#                ,'total invoiced in last 3 months vs prior 3 months'
#                ,'avg mthly num of invoiced items in last 6 months vs prior 6 months'
#                ,'avg mthly num of invoiced items in last 6 months'
#                ,'number of device_status = online/complete'
               
               #features that had total gain less than 3.0
,'does account have - rpc core'
,'rating_passive_to_passive_num'
,'num opps category - cloud net revenue'
,'rating_promoter_to_passive_num'
,'rating_passive_to_promoter_num'
,'num opps requested_products - hosting only'
,'how many units - rpc core'
,'does account have - faws'
,'opp_in_stage1_cnt_change_from_last_month'
,'num tickets ticket category - monitoring 1mo'
,'num tickets ticket category - monitoring alerts 1mo'
,'num opps typex - aws'
,'does account have - threat manager'
,'num distinct account_geographic_location'
,'avg minutes to first racker comment - category monitoring alerts 1mo'
,'opp_in_stage1_cnt'
,'avg minutes to started in progress - category monitoring alerts 1mo'
,'does account have - san'
,'num distinct account_business_type'
,'does account have - virtualization'
,'opp_in_stage3_cnt_change_from_last_month'
,'num distinct account_billing_country'
,'num opps data_quality_score - 100'
,'does account have - managed storage'
,'num distinct account_primary_contact'
,'pct of invoice - rpc core'
,'revenue_tickets_closed_last1mos'
,'num opps typex - mail contract signup'
,'avg days to close ticket - category customer initiated 1mo'
,'num distinct account_bdc'
,'num opportunities lost last 6 months'
,'num opportunities last 12 months'
,'num distinct account_team_name'
,'num opps focus_area - openstack public'
,'avg days to close ticket 1mo'
,'num opps leadsource - site submission'
,'does account have - bandwidth overages'
,'num opps leadsource - call in'
,'num opps focus_area - amazon'
,'num distinct account_billing_state'
,'does account have - rackspace email'
,'num tickets ticket category - application 1mo'
,'num distinct account_billing_city'
,'num opps contract_length - 1'
,'num opportunities lost last 3 months'
,'per unit price - rpc core'
,'opp_is_revenue_ticket_cnt_change_from_last_month'
,'does account have - load balancer'
,'num opps allow_quote - allowed'
,'num opps leadsource - partner network'
,'num tickets severity - emergency 1mo'
,'num opps forecastcategoryname - omitted'
,'how many units - managed exchange'
,'avg minutes to first racker comment - category monitoring alerts'
,'num opps category - new'
,'avg minutes to first racker comment 1mo'
,'num opps support_unit - smb'
,'rate of tickets confirmed solved 1mo'
,'avg days to close ticket - category application 1mo'
,'num tickets ticket category - operating services 1mo'
,'avg days to close ticket - all customer initiated tickets 1mo'
,'per unit price - virtual hosting'
,'num opps support_unit - email & apps'
,'avg days to close ticket - all racker initiated tickets 1mo'
,'num opps forecastcategoryname - pipeline'
,'avg minutes to first customer comment 1mo'
,'avg minutes to first racker comment'
,'num opps contract_length - 12'
,'num tickets ticket category - operating services'
,'num opps data_quality_description - missing: lead source'
,'num opps category - new logo'
,'num opps cvp_verified - true'
,'avg minutes to started in progress'
,'num opps probability - 0'
,'pct of devices with contract_status = in contract - risk of lapse 90 days'
,'num opps on_demand_reconciled - false'
,'num tickets ticket category - monitoring alerts'
,'avg days to close ticket - category hardware'
,'does account have - ppc'
,'num tickets ticket category - monitoring'
,'acc_tenure_in_months'
,'num opps data_quality_description - missing: next steps'
,'pct of revenue with contract_status = no contract status'
,'num distinct account_billing_postal_code'
,'does account have - mspc'
,'does account have - vmware'
,'avg days to close ticket - category customer initiated'
,'rate of tickets solved 1mo'
,'rating_promoter_to_promoter_num'
,'num opps focus_area - cloud office'
,'avg days to close ticket - category hardware 1mo'
,'does account have - cloud office'
,'avg days to close ticket'
,'num opps bucket_influence - marketing'
,'opp_is_revenue_ticket_cnt'
,'does account have - cloud san'
,'does account have - virtual hosting'
,'account_sub_type external customer'
,'how many units - cloud san'
,'does account have - switch'
,'avg minutes to started in progress 1mo'
,'avg days to close ticket - category application'
,'total number of tickets'
,'number of device_status = support maintenance'
,'num distinct account_billing_street'
,'num opps focus_area - cloud'
,'num tickets ticket category - account management 1mo'
,'num opps forecastcategory - omitted'
,'number of tickets solved 1mo'
,'num opps nutcase_deal_probability - 0'
,'per unit price - managed storage'
,'num opps cvp_verified - false'
,'num distinct account_manager'
,'num opps category - new footprint'
,'num tickets ticket category - application'
,'avg days to close ticket - all customer initiated tickets'
,'num opps data_quality_description - missing: amount'
,'per unit price - switch'
,'num opps data_quality_score - 60'
,'num opps probability - 100'
,'num opps market_source - yes'
,'num opps commission_role - pay commissions'
,'pct of invoice - switch'
,'rate of tickets confirmed solved'
,'num tickets ticket category - customer initiated 1mo'
,'num opps data_quality_score - 80'
,'num opps econnect_received - false'
,'second_last_survey_score'
,'pct of invoice - virtualization'
,'rate of tickets solved'
,'num tickets ticket category - account management'
,'pct of revenue with contract_status = in contract - risk of lapse 90 days'
,'number of tickets solved'
,'how many units - threat manager'
,'num opps data_quality_description - missing: amount, lead source'
,'num opps contract_length - 24'
,'num opps why_did_we_lose - project abandoned'
,'avg minutes to started in progress - category monitoring alerts'
,'number of devices os - firewall'
,'num tickets severity - emergency'
,'num opps contract_length - 0'
,'pct of invoice - managed storage'
,'avg minutes to started in progress - severity urgent 1mo'
,'num opps data_quality_description - all opportunity details captur'
,'does account have - managed exchange'
,'account_sub_type other'
,'per unit price - cloud san'
,'avg monthly number of - sku_description =  privatenet - 6 mth to 1 mth'
,'does account have - firewall'
,'num opps category - migration'
,'last_survey_score'
,'avg minutes to started in progress - severity urgent'
,'num opps leadsource - outbound'
,'num opps why_did_we_lose - price'
,'pct of invoice - load balancer'
,'num opps forecastcategory - pipeline'
,'num opps bucket_source - sales'
,'num distinct account_region'
,'num tickets ticket category - customer initiated'
,'num opps support_unit - enterprise'
,'num opportunities last 6 months'
,'avg days to close ticket - all racker initiated tickets'
,'num opps data_quality_description - missing: amount, next steps'
,'how many units - switch'
,'num opps ticket_type - upgrade'
,'number of device_status = online/complete'
,'num opps category - upgrade'
,'avg per line item invoiced in last 6 months_original'
               
               
               #2nd round of features
,'avg minutes to first customer comment'
,'avg monthly number of - sku_name =  hard drive size - 6 mth to 1 mth'
,'avg monthly number of - sku_name =  high availability - 6 mth to 1 mth'
,'avg monthly number of - sku_name =  load-balancer required - 6 mth to 1 mth'
,'does account have - server'
,'how many units - firewall'
,'how many units - managed storage'
,'how many units - rackspace email'
,'longest active device tenure months'
,'num opps allow_quote - not allowed'
,'num opps bucket_source - marketing'
,'num opps data_quality_description - missing: lead source, next ste'
,'num opps data_quality_score - 40'
,'num opps forecastcategory - closed'
,'num opps iswon - false'
,'num opps iswon - true'
,'num opps leadsource - chat'
,'num opps leadsource - null'
,'num opps live_call - false'
,'num opps market_source - no'
,'num opps pain_point - servicenow'
,'num opps typex - cloud/private cloud'
,'num opps typex - rackspace cloud'
,'num opps typex - revenue ticket'
,'num opps what_did_we_do_well - solution fit'
,'num opps why_did_we_lose - existing opp/closed via ticket'
,'num opps why_did_we_lose - no response'
,'number of devices os - load-balancer'
,'number of devices os - switch'
,'number of tickets confirmed solved'
,'opp_in_stage3_cnt'
,'pct of device_status = online/complete'
,'pct of device_status = support maintenance'
,'pct of devices with contract_status = no contract status'
,'pct of invoice - cloud san'
,'pct of invoice - firewall'
,'pct of invoice - server'
,'pct of invoice - virtual hosting'
,'pct of revenue with contract_status = in contract'
,'pct of revenue with contract_status = out of contract - mtm'
,'per unit price - bandwidth overages'
,'per unit price - load balancer'
,'per unit price - managed exchange'
,'per unit price - san'
,'per unit price - server'
,'per unit price - virtualization'
,'rating_detractor_to_passive_num'
,'rating_detractor_to_promoter_num'
,'rating_passive_to_detractor_num'
,'rating_promoter_to_detractor_num'

              ]
include_col = ['opp_in_stage1_cnt'
    ,'opp_in_stage3_cnt'
    ,'opp_in_contract_state_cnt'
    ,'multi_stage_in_month_cnt'
    ,'opp_is_revenue_ticket_cnt'
    ,'opp_in_stage1_cnt_change_from_last_month'
    ,'opp_in_stage3_cnt_change_from_last_month'
    ,'opp_is_revenue_ticket_cnt_change_from_last_month'
    ,'website_chats_by_tmk'
    ,'website_calls_by_tmk'
    ,'website_sitesubs_by_tmk'
    ,'website_marketo_form_subs_by_tmk'
    ,'website_chats_by_tmk_change_from_last_month'
    ,'website_calls_by_tmk_change_from_last_month'
    ,'website_sitesubs_by_tmk_change_from_last_month'
    ,'website_marketo_form_subs_by_tmk_change_from_last_month'
    ,'avg credit memo in last 6 months'
    ,'total invoiced in last 3 months vs prior 3 months'
    ,'avg mthly num of invoiced items in last 3 months vs prior 3 months'
    ,'total number of tickets'
    ,'avg minutes to first customer comment'
    ,'avg minutes to first racker comment'
    ,'number of tickets confirmed solved'
    ,'number of tickets solved'
    ,'rate of tickets confirmed solved'
    ,'rate of tickets solved'
    ,'last_survey_score'
    ,'second_last_survey_score'
    ,'num opportunities last 6 months'
    ,'num opportunities won last 6 months'
    ,'num opportunities lost last 6 months'
    ,'pct opportunities won last 6 months'
    ,'pct opportunities lost last 6 months'
    ,'num opportunities last 3 months'
    ,'num opportunities won last 3 months'
    ,'num opportunities lost last 3 months'
    ,'pct opportunities won last 3 months'
    ,'pct opportunities lost last 3 months'
    ,'total value of opportunities last 6 months'
    ,'total value of opportunities last 3 months'
    ,'average value of opportunities last 6 months'
    ,'average value of opportunities last 3 months'
               ]
predictors = list(cloud_train.columns[~cloud_train.columns.isin(exclude_col)])
#predictors = list(cloud_num.columns[cloud_num.columns.isin(include_col)])

#create copy of the X and Y columns
x_low = cloud_train_low[predictors] #uses list of columns excluding target (predictors) to select columns for x_low
y1_low = cloud_train_low['target']

x_mid = cloud_train_mid[predictors]
y1_mid = cloud_train_mid['target']

x_hi = cloud_train_hi[predictors]
y1_hi = cloud_train_hi['target']

print("train ready")

#create copy of the X and Y columns for eval data
x_eval_low = cloud_eval_low[predictors] #uses list of columns excluding target (predictors) to select columns for x_low
y1_eval_low = cloud_eval_low['target']

x_eval_mid = cloud_eval_mid[predictors]
y1_eval_mid = cloud_eval_mid['target']

x_eval_hi = cloud_eval_hi[predictors]
y1_eval_hi = cloud_eval_hi['target']

print("eval ready")

#==================================================================================================================
#==================================================================================================================
# Resample minority class in training dataset
#==================================================================================================================
#==================================================================================================================

seed = 178

ros_low = RandomUnderSampler(random_state=seed)
x1_train_res_low0, y1_train_res_low0 = ros_low.fit_sample(x_low, y1_low)
x1_train_res_low = pd.DataFrame(x1_train_res_low0)
x1_train_res_low.columns = predictors
y1_train_res_low = pd.DataFrame(y1_train_res_low0)

ros_mid = RandomUnderSampler(random_state=seed)
x1_train_res_mid0, y1_train_res_mid0 = ros_mid.fit_sample(x_mid, y1_mid)
x1_train_res_mid = pd.DataFrame(x1_train_res_mid0)
x1_train_res_mid.columns = predictors
y1_train_res_mid = pd.DataFrame(y1_train_res_mid0)

ros_hi = RandomUnderSampler(random_state=seed)
x1_train_res_hi0, y1_train_res_hi0 = ros_hi.fit_sample(x_hi, y1_hi)
x1_train_res_hi = pd.DataFrame(x1_train_res_hi0)
x1_train_res_hi.columns = predictors
y1_train_res_hi = pd.DataFrame(y1_train_res_hi0)

print('Minority class resampling complete')

#rerun model using same parameters as before
p.modeleval(xgb_final_low, x1_train_res_low, y1_train_res_low.values.ravel(), 
         x_eval_low, y1_eval_low, predictors)
p.modeleval(xgb_final_mid, x1_train_res_mid, y1_train_res_mid.values.ravel(), 
         x_eval_mid, y1_eval_mid, predictors)
p.modeleval(xgb_final_hi, x1_train_res_hi, y1_train_res_hi.values.ravel(),
         x_eval_hi, y1_eval_hi, predictors)

In [ ]:
#Available importance_types = ['weight', 'gain', 'cover', 'total_gain', 'total_cover']
#'gain' is the average GINI gain for that feature when used in a split
#re-run for the second iteration of eliminated features
fi_low_gain = pd.DataFrame(xgb_final_low.get_booster().get_score(importance_type= 'gain'),index=[0]) #have to specify index to satisfy conversion to dataframe
fi_low_gain = fi_low_gain.T #transpose result
fi_low_gain = fi_low_gain.sort_values([0], ascending=[True]).copy()
fi_low_gain.to_excel('3feat_imp_low.xlsx')

fi_mid_gain = pd.DataFrame(xgb_final_mid.get_booster().get_score(importance_type= 'gain'),index=[0]) #have to specify index to satisfy conversion to dataframe
fi_mid_gain = fi_mid_gain.T #transpose result
fi_mid_gain = fi_mid_gain.sort_values([0], ascending=[True]).copy()
fi_mid_gain.to_excel('3feat_imp_mid.xlsx')

fi_hi_gain = pd.DataFrame(xgb_final_hi.get_booster().get_score(importance_type= 'gain'),index=[0]) #have to specify index to satisfy conversion to dataframe
fi_hi_gain = fi_hi_gain.T #transpose result
fi_hi_gain = fi_hi_gain.sort_values([0], ascending=[True]).copy()
fi_hi_gain.to_excel('3feat_imp_hi.xlsx')


In [ ]:
#3rd round of elimination
#based on list of features with gain <3.0 for each revenue band, eliminate those features from the dataset by rev band, 
#rerun the model

#examination of features from mid and low model for features with importance gain <3.0 show that their lists have features
#that are almost all in the elimination list from the high revenue accounts
#therefore, for simplicity in the first round, we'll eliminate the features from the high revenue accounts, as this list
#is much larger than the list from the mid and low rev. We'll check the model fit results and see if there's a negative impact

include_col_hi = ['3 mth pct change number of device_status = computer no longer active'
,'3 mth pct change number of devices active status - active'
,'3 mth pct change number of devices online status - online'
,'average active device tenure months'
,'average value of opportunities last 3 months'
,'average value of opportunities last 6 months'
,'avg credit memo in last 6 months'
,'avg monthly number of - sku_description =  included bandwidth - 6 mth to 1 mth'
,'avg monthly number of - sku_description =  raid 1 - 6 mth to 1 mth'
,'avg monthly number of - sku_name =  included bandwidth - 6 mth to 1 mth'
,'avg monthly number of - sku_name =  ip allocation - 6 mth to 1 mth'
,'avg mthly num of invoiced items in last 3 months vs prior 3 months'
,'avg mthly num of invoiced items in last 6 months'
,'how many units - bandwidth overages'
,'how many units - san'
,'how many units - server'
,'multi_stage_in_month_cnt'
,'num opportunities last 3 months'
,'num opportunities won last 3 months'
,'num opportunities won last 6 months'
,'num opps competitors - other'
,'num opps econnect_received - true'
,'num opps on_demand_reconciled - true'
,'num tickets ticket category - null 1mo'
,'number of device_status = computer no longer active'
,'number of other device_status'
,'pct of device_status = computer no longer active'
,'pct of invoice - bandwidth overages'
,'pct of invoice - rackspace email'
,'pct of other device_status'
,'pct opportunities lost last 3 months'
,'pct opportunities lost last 6 months'
,'pct opportunities won last 3 months'
,'pct opportunities won last 6 months'
,'per unit price - firewall'
,'per unit price - threat manager'
,'rating_detractor_to_detractor_num'
,'shortest active device tenure months'
,'total invoiced in last 3 months vs prior 3 months'
,'total value of opportunities last 3 months'
,'total value of opportunities last 6 months'
               ]

include_col_mid = [
'3 mth pct change longest active device tenure months'
,'3 mth pct change number of device_status = computer no longer active'
,'3 mth pct change number of devices active status - active'
,'3 mth pct change number of devices online status - online'
,'average active device tenure months'
,'average value of opportunities last 3 months'
,'average value of opportunities last 6 months'
,'avg credit memo in last 6 months'
,'avg monthly number of - sku_description =  included bandwidth - 6 mth to 1 mth'
,'avg monthly number of - sku_description =  raid 1 - 6 mth to 1 mth'
,'avg monthly number of - sku_name =  advanced_networking - 6 mth to 1 mth'
,'avg monthly number of - sku_name =  included bandwidth - 6 mth to 1 mth'
,'avg monthly number of - sku_name =  ip allocation - 6 mth to 1 mth'
,'avg mthly num of invoiced items in last 3 months vs prior 3 months'
,'avg mthly num of invoiced items in last 6 months'
,'how many units - bandwidth overages'
,'how many units - load balancer'
,'how many units - san'
,'how many units - server'
,'how many units - virtual hosting'
,'how many units - virtualization'
,'multi_stage_in_month_cnt'
,'num opportunities won last 3 months'
,'num opportunities won last 6 months'
,'num opps competitors - in-house'
,'num opps competitors - other'
,'num opps econnect_received - true'
,'num opps forecastcategoryname - closed'
,'num opps on_demand_reconciled - true'
,'num opps pain_point - other'
,'num tickets ticket category - null'
,'number of device_status = computer no longer active'
,'number of devices last month'
,'number of devices os name - linux'
,'number of other device_status'
,'number of tickets confirmed solved 1mo'
,'opp_in_contract_state_cnt'
,'pct of device_status = computer no longer active'
,'pct of devices with contract_status = in contract'
,'pct of devices with contract_status = out of contract - mtm'
,'pct of invoice - bandwidth overages'
,'pct of invoice - managed exchange'
,'pct of invoice - rackspace email'
,'pct of invoice - san'
,'pct of invoice - threat manager'
,'pct of other device_status'
,'pct opportunities lost last 3 months'
,'pct opportunities lost last 6 months'
,'pct opportunities won last 3 months'
,'pct opportunities won last 6 months'
,'per unit price - firewall'
,'per unit price - rackspace email'
,'per unit price - threat manager'
,'rating_detractor_to_detractor_num'
,'shortest active device tenure months'
,'total invoiced in last 3 months vs prior 3 months'
,'total number of tickets 1mo'
,'total value of opportunities last 3 months'
,'total value of opportunities last 6 months'
]

include_col_low = [
'3 mth pct change number of device_status = computer no longer active'
,'3 mth pct change number of devices active status - active'
,'3 mth pct change number of devices online status - online'
,'average value of opportunities last 3 months'
,'average value of opportunities last 6 months'
,'avg credit memo in last 6 months'
,'avg monthly number of - sku_name =  ip allocation - 6 mth to 1 mth'
,'how many units - bandwidth overages'
,'how many units - san'
,'how many units - server'
,'multi_stage_in_month_cnt'
,'num opportunities won last 3 months'
,'num opportunities won last 6 months'
,'num opps on_demand_reconciled - true'
,'num opps pain_point - other'
,'number of device_status = computer no longer active'
,'number of devices os name - linux'
,'number of other device_status'
,'opp_in_contract_state_cnt'
,'pct of device_status = computer no longer active'
,'pct of invoice - bandwidth overages'
,'pct of invoice - managed exchange'
,'pct of invoice - rackspace email'
,'pct of other device_status'
,'pct opportunities lost last 3 months'
,'pct opportunities lost last 6 months'
,'pct opportunities won last 3 months'
,'pct opportunities won last 6 months'
,'per unit price - firewall'
,'per unit price - rackspace email'
,'per unit price - threat manager'
,'rating_detractor_to_detractor_num'
,'shortest active device tenure months'
,'total invoiced in last 3 months vs prior 3 months'
,'total value of opportunities last 3 months'
,'total value of opportunities last 6 months'
]
#predictors = list(cloud_train.columns[~cloud_train.columns.isin(exclude_col)])
predictors_hi = list(cloud_train.columns[cloud_train.columns.isin(include_col_hi)])
predictors_mid = list(cloud_train.columns[cloud_train.columns.isin(include_col_mid)])
predictors_low = list(cloud_train.columns[cloud_train.columns.isin(include_col_low)])

#create copy of the X and Y columns
x_low = cloud_train_low[predictors_low] #uses list of columns excluding target (predictors) to select columns for x_low
y1_low = cloud_train_low['target']

x_mid = cloud_train_mid[predictors_mid]
y1_mid = cloud_train_mid['target']

x_hi = cloud_train_hi[predictors_hi]
y1_hi = cloud_train_hi['target']

print("train ready")

#create copy of the X and Y columns for eval data
x_eval_low = cloud_eval_low[predictors_low] #uses list of columns excluding target (predictors) to select columns for x_low
y1_eval_low = cloud_eval_low['target']

x_eval_mid = cloud_eval_mid[predictors_mid]
y1_eval_mid = cloud_eval_mid['target']

x_eval_hi = cloud_eval_hi[predictors_hi]
y1_eval_hi = cloud_eval_hi['target']

print("eval ready")

#==================================================================================================================
#==================================================================================================================
# Resample minority class in training dataset
#==================================================================================================================
#==================================================================================================================

seed = 178

ros_low = RandomUnderSampler(random_state=seed)
x1_train_res_low0, y1_train_res_low0 = ros_low.fit_sample(x_low, y1_low)
x1_train_res_low = pd.DataFrame(x1_train_res_low0)
x1_train_res_low.columns = predictors_low
y1_train_res_low = pd.DataFrame(y1_train_res_low0)

ros_mid = RandomUnderSampler(random_state=seed)
x1_train_res_mid0, y1_train_res_mid0 = ros_mid.fit_sample(x_mid, y1_mid)
x1_train_res_mid = pd.DataFrame(x1_train_res_mid0)
x1_train_res_mid.columns = predictors_mid
y1_train_res_mid = pd.DataFrame(y1_train_res_mid0)

ros_hi = RandomUnderSampler(random_state=seed)
x1_train_res_hi0, y1_train_res_hi0 = ros_hi.fit_sample(x_hi, y1_hi)
x1_train_res_hi = pd.DataFrame(x1_train_res_hi0)
x1_train_res_hi.columns = predictors_hi
y1_train_res_hi = pd.DataFrame(y1_train_res_hi0)

print('Minority class resampling complete')

#rerun model using same parameters as before
p.modeleval(xgb_final_low, x1_train_res_low, y1_train_res_low.values.ravel(), 
         x_eval_low, y1_eval_low, predictors_low)
p.modeleval(xgb_final_mid, x1_train_res_mid, y1_train_res_mid.values.ravel(), 
         x_eval_mid, y1_eval_mid, predictors_mid)
p.modeleval(xgb_final_hi, x1_train_res_hi, y1_train_res_hi.values.ravel(),
         x_eval_hi, y1_eval_hi, predictors_hi)

In [ ]:
#rerun again but just use hi rev feature list, for simplicity, and see how results come out

In [ ]:
#3rd round of elimination
#based on list of features with gain <3.0 for each revenue band, eliminate those features from the dataset by rev band, 
#rerun the model

#examination of features from mid and low model for features with importance gain <3.0 show that their lists have features
#that are almost all in the elimination list from the high revenue accounts
#therefore, for simplicity in the first round, we'll eliminate the features from the high revenue accounts, as this list
#is much larger than the list from the mid and low rev. We'll check the model fit results and see if there's a negative impact

include_col = ['3 mth pct change number of device_status = computer no longer active'
,'3 mth pct change number of devices active status - active'
,'3 mth pct change number of devices online status - online'
,'average active device tenure months'
,'average value of opportunities last 3 months'
,'average value of opportunities last 6 months'
,'avg credit memo in last 6 months'
,'avg monthly number of - sku_description =  included bandwidth - 6 mth to 1 mth'
,'avg monthly number of - sku_description =  raid 1 - 6 mth to 1 mth'
,'avg monthly number of - sku_name =  included bandwidth - 6 mth to 1 mth'
,'avg monthly number of - sku_name =  ip allocation - 6 mth to 1 mth'
,'avg mthly num of invoiced items in last 3 months vs prior 3 months'
,'avg mthly num of invoiced items in last 6 months'
,'how many units - bandwidth overages'
,'how many units - san'
,'how many units - server'
,'multi_stage_in_month_cnt'
,'num opportunities last 3 months'
,'num opportunities won last 3 months'
,'num opportunities won last 6 months'
,'num opps competitors - other'
,'num opps econnect_received - true'
,'num opps on_demand_reconciled - true'
,'num tickets ticket category - null 1mo'
,'number of device_status = computer no longer active'
,'number of other device_status'
,'pct of device_status = computer no longer active'
,'pct of invoice - bandwidth overages'
,'pct of invoice - rackspace email'
,'pct of other device_status'
,'pct opportunities lost last 3 months'
,'pct opportunities lost last 6 months'
,'pct opportunities won last 3 months'
,'pct opportunities won last 6 months'
,'per unit price - firewall'
,'per unit price - threat manager'
,'rating_detractor_to_detractor_num'
,'shortest active device tenure months'
,'total invoiced in last 3 months vs prior 3 months'
,'total value of opportunities last 3 months'
,'total value of opportunities last 6 months'
               ]

#predictors = list(cloud_train.columns[~cloud_train.columns.isin(exclude_col)])
predictors = list(cloud_train.columns[cloud_train.columns.isin(include_col)])


#create copy of the X and Y columns
x1_train_res_low = cloud_train_low_rus[predictors] #uses list of columns excluding target (predictors) to select columns for x_low
y1_train_res_low = cloud_train_low_rus['target']

x1_train_res_mid = cloud_train_mid_rus[predictors]
y1_train_res_mid = cloud_train_mid_rus['target']

x1_train_res_hi = cloud_train_hi_rus[predictors]
y1_train_res_hi = cloud_train_hi_rus['target']

print("train ready")

#create copy of the X and Y columns for eval data
x_eval_low = cloud_eval_low[predictors] #uses list of columns excluding target (predictors) to select columns for x_low
y1_eval_low = cloud_eval_low['target']

x_eval_mid = cloud_eval_mid[predictors]
y1_eval_mid = cloud_eval_mid['target']

x_eval_hi = cloud_eval_hi[predictors]
y1_eval_hi = cloud_eval_hi['target']

print("eval ready")

print("eval ready")


print('Minority class resampling complete')

#rerun model using same parameters as before
p.modeleval(xgb_final_low, x1_train_res_low, y1_train_res_low.values.ravel(), 
         x_eval_low, y1_eval_low, predictors)
p.modeleval(xgb_final_mid, x1_train_res_mid, y1_train_res_mid.values.ravel(), 
         x_eval_mid, y1_eval_mid, predictors)
p.modeleval(xgb_final_hi, x1_train_res_hi, y1_train_res_hi.values.ravel(),
         x_eval_hi, y1_eval_hi, predictors)

In [ ]:
#rerun model using same parameters as before
p.modeleval(xgb_final_low, x1_train_res_low, y1_train_res_low.values.ravel(), 
         x_eval_low, y1_eval_low, predictors)
p.modeleval(xgb_final_mid, x1_train_res_mid, y1_train_res_mid.values.ravel(), 
         x_eval_mid, y1_eval_mid, predictors)
p.modeleval(xgb_final_hi, x1_train_res_hi, y1_train_res_hi.values.ravel(),
         x_eval_hi, y1_eval_hi, predictors)

In [ ]:
# create most recent feature importance plot for latest model : gbm_final_low, gbm_final_mid, gbm_final_hi
import matplotlib.pyplot as pyplot
from xgboost import plot_importance

rcParams['figure.figsize'] = 12, 12
fi_low = pd.concat([pd.DataFrame(predictors),pd.DataFrame(xgb_final_low.feature_importances_)], axis=1)
fi_low.columns=['features','feature importance']
fi_mid = pd.concat([pd.DataFrame(predictors),pd.DataFrame(xgb_final_mid.feature_importances_)], axis=1)
fi_mid.columns=['features','feature importance']
fi_hi = pd.concat([pd.DataFrame(predictors),pd.DataFrame(xgb_final_hi.feature_importances_)], axis=1)
fi_hi.columns=['features','feature importance']


# High Value Accounts
#print(fi_hi)
print(xgb_final_hi)

print('\nfeature importance chart - hi')
plot_importance(xgb_final_hi)
pyplot.show()
#print('feature importance chart- cover')
#plot_importance(xgb_final_hi, importance_type="cover")
pyplot.show()
print('feature importance chart- gain')
plot_importance(xgb_final_hi, importance_type="gain")
pyplot.show()

#Mid Value Accounts
#print(fi_mid)
print(xgb_final_mid)

print('\nfeature importance chart - mid')
plot_importance(xgb_final_mid)
pyplot.show()
#print('feature importance chart- cover')
#plot_importance(xgb_final_mid, importance_type="cover")
pyplot.show()
print('feature importance chart- gain')
plot_importance(xgb_final_mid, importance_type="gain")
pyplot.show()

#Low Value Accounts
#print(fi_low)
print(xgb_final_low)

import matplotlib.pyplot as pyplot

print('\nfeature importance chart - low')
plot_importance(xgb_final_low)
pyplot.show()
#print('feature importance chart- cover')
#plot_importance(xgb_final_low, importance_type="cover")
pyplot.show()
print('feature importance chart- gain')
plot_importance(xgb_final_low, importance_type="gain")
pyplot.show()

In [ ]:
from xgboost import plot_tree
plot_tree(xgb_final_hi) #this feature won't work if white spaces are in the feature names.. which are in almost all of them